In [19]:
import pandas as pd

df = pd.read_csv('..\\asset_group/NT2_23.TXT', delimiter='"\t"', engine='python')
correction = pd.read_excel('..\\asset_group/correction_check.xlsx', skiprows=7, usecols='D:I', nrows=6)
asset_group = pd.read_csv('..\\asset_group/BU_ASSET_GROUP_ADD_INTANGIBLT_NT2_.csv', dtype=str)

ดึง " ที่เหลืออยู่ออก

In [20]:
df.columns = [col.strip('"') for col in df.columns]
df = df.rename(columns=lambda x: x.strip())
df = df.map(lambda x: x.strip('"') if isinstance(x, str) else x)

โชว์ข้อมูลที่ผิดพลาด

In [12]:
df2 = pd.read_csv('..\\asset_group/NT2_23.TXT', delimiter='\t', dtype=str)

df2[' ACQUISITION_VALUE'] = df2[' ACQUISITION_VALUE'].astype(str).str.strip()
corrupt_df = df2[~df2[' ACQUISITION_VALUE'].str.match(r'^\d+(\.\d+)?$', na=False)]
corrupt_df.to_csv('..\\asset_group/result/corrupt_data.csv')

นำออกไฟล์ที่แก้ไขข้อผิดพลาดแล้ว

In [13]:
df.to_csv('..\\asset_group/result/NT2_23_FIX.csv', index=False)

ยอดรวมทรัพย์สิน

In [21]:
sum_df = pd.DataFrame({
    ' ':['ที่ดินอาคารอุปกรณ์ Total', 'สินทรัพย์ไม่มีตัวตน Total', 'อสังหาริมทรัพย์ฯ Total', 'สินทรัพย์สัญญาเช่า Total', 'สินทรัพย์ระหว่างก่อสร้าง', 'ยอดรวมสินทรัพย์ทั้งสิ้น'],})

sum_df[['มูลค่าการได้มา','ค่าเสื่อมราคาสะสม','ค่าเผื่อการด้อยค่า','มูลค่าตามบัญชี','ค่าเสื่อมเดือน','ค่าเสื่อมราคาในปี']] = None

รวมยอดทรัพย์สินทั้งหมด

In [22]:
land = asset_group.loc[(asset_group['BU_ASSET_GROUP'].str.contains('ที่ดิน อาคาร อุปกรณ์')) & (~asset_group['ACCOUNTING_NAME'].str.contains('สินทรัพย์ระหว่างก่อสร้าง'))]
land = land['ACCOUNTING_NUMBER']

Intangible = asset_group.loc[asset_group['BU_ASSET_GROUP'].str.contains('สินทรัพย์ไม่มีตัวตน')]
Intangible = Intangible['ACCOUNTING_NUMBER']

realty = asset_group.loc[asset_group['BU_ASSET_GROUP'].str.contains('อสังหาริมทรัพย์เพื่อการลงทุน')]
realty = realty['ACCOUNTING_NUMBER']

rent = asset_group.loc[asset_group['BU_ASSET_GROUP'].str.contains('สิทธิการเช่าใช้')]
rent = rent['ACCOUNTING_NUMBER']

construction = asset_group.loc[asset_group['ACCOUNTING_NAME'].str.contains('สินทรัพย์ระหว่างก่อสร้าง')]
construction = construction['ACCOUNTING_NUMBER']

In [23]:
df_land = pd.merge(df, land, how='right', on='ACCOUNTING_NUMBER')
sum_df.at[0,'มูลค่าการได้มา'] = df_land['ACQUISITION_VALUE'].sum()
sum_df.at[0,'ค่าเสื่อมราคาสะสม'] = df_land['ACCUMULATED_DEPRECIATION'].sum()
sum_df.at[0,'ค่าเผื่อการด้อยค่า'] = df_land['IMPAIRMENT_OF_ASSET'].sum()
sum_df.at[0,'มูลค่าตามบัญชี'] = df_land['BOOK_VALUE'].sum()
sum_df.at[0,'ค่าเสื่อมเดือน'] = df_land['DEPRECIATION_OF_MONTH'].sum()
sum_df.at[0,'ค่าเสื่อมราคาในปี'] = df_land['DEPRECIATION_YEAREND'].sum()

df_intangible = pd.merge(df, Intangible, how='right', on='ACCOUNTING_NUMBER')
sum_df.at[1,'มูลค่าการได้มา'] = df_intangible['ACQUISITION_VALUE'].sum()
sum_df.at[1,'ค่าเสื่อมราคาสะสม'] = df_intangible['ACCUMULATED_DEPRECIATION'].sum()
sum_df.at[1,'ค่าเผื่อการด้อยค่า'] = df_intangible['IMPAIRMENT_OF_ASSET'].sum()
sum_df.at[1,'มูลค่าตามบัญชี'] = df_intangible['BOOK_VALUE'].sum()
sum_df.at[1,'ค่าเสื่อมเดือน'] = df_intangible['DEPRECIATION_OF_MONTH'].sum()
sum_df.at[1,'ค่าเสื่อมราคาในปี'] = df_intangible['DEPRECIATION_YEAREND'].sum()

df_realty = pd.merge(df, realty, how='right', on='ACCOUNTING_NUMBER')
sum_df.at[2,'มูลค่าการได้มา'] = df_realty['ACQUISITION_VALUE'].sum()
sum_df.at[2,'ค่าเสื่อมราคาสะสม'] = df_realty['ACCUMULATED_DEPRECIATION'].sum()
sum_df.at[2,'ค่าเผื่อการด้อยค่า'] = df_realty['IMPAIRMENT_OF_ASSET'].sum()
sum_df.at[2,'มูลค่าตามบัญชี'] = df_realty['BOOK_VALUE'].sum()
sum_df.at[2,'ค่าเสื่อมเดือน'] = df_realty['DEPRECIATION_OF_MONTH'].sum()
sum_df.at[2,'ค่าเสื่อมราคาในปี'] = df_realty['DEPRECIATION_YEAREND'].sum()

df_rent = pd.merge(df, rent, how='right', on='ACCOUNTING_NUMBER')
sum_df.at[3,'มูลค่าการได้มา'] = df_rent['ACQUISITION_VALUE'].sum()
sum_df.at[3,'ค่าเสื่อมราคาสะสม'] = df_rent['ACCUMULATED_DEPRECIATION'].sum()
sum_df.at[3,'ค่าเผื่อการด้อยค่า'] = df_rent['IMPAIRMENT_OF_ASSET'].sum()
sum_df.at[3,'มูลค่าตามบัญชี'] = df_rent['BOOK_VALUE'].sum()
sum_df.at[3,'ค่าเสื่อมเดือน'] = df_rent['DEPRECIATION_OF_MONTH'].sum()
sum_df.at[3,'ค่าเสื่อมราคาในปี'] = df_rent['DEPRECIATION_YEAREND'].sum()

df_constuction = pd.merge(df, construction, how='right', on='ACCOUNTING_NUMBER')
sum_df.at[4,'มูลค่าการได้มา'] = df_constuction['ACQUISITION_VALUE'].sum()
sum_df.at[4,'ค่าเสื่อมราคาสะสม'] = df_constuction['ACCUMULATED_DEPRECIATION'].sum()
sum_df.at[4,'ค่าเผื่อการด้อยค่า'] = df_constuction['IMPAIRMENT_OF_ASSET'].sum()
sum_df.at[4,'มูลค่าตามบัญชี'] = df_constuction['BOOK_VALUE'].sum()
sum_df.at[4,'ค่าเสื่อมเดือน'] = df_constuction['DEPRECIATION_OF_MONTH'].sum()
sum_df.at[4,'ค่าเสื่อมราคาในปี'] = df_constuction['DEPRECIATION_YEAREND'].sum()

sum_df.at[5,'มูลค่าการได้มา'] = df['ACQUISITION_VALUE'].sum()
sum_df.at[5,'ค่าเสื่อมราคาสะสม'] = df['ACCUMULATED_DEPRECIATION'].sum()
sum_df.at[5,'ค่าเผื่อการด้อยค่า'] = df['IMPAIRMENT_OF_ASSET'].sum()
sum_df.at[5,'มูลค่าตามบัญชี'] = df['BOOK_VALUE'].sum()
sum_df.at[5,'ค่าเสื่อมเดือน'] = df['DEPRECIATION_OF_MONTH'].sum()
sum_df.at[5,'ค่าเสื่อมราคาในปี'] = df['DEPRECIATION_YEAREND'].sum()

sum_df['ค่าเสื่อมเดือน'] = sum_df['ค่าเสื่อมเดือน'].map(lambda x: -x)
sum_df['ค่าเสื่อมราคาในปี'] = sum_df['ค่าเสื่อมราคาในปี'].map(lambda x: -x)

In [24]:
sum_df.to_excel('..\\asset_group/result/total_result.xlsx', index=False)

ตรวจสอบความถูกต้อง

In [25]:
sum_df = sum_df.drop(sum_df.columns[0], axis=1)
sum_df = sum_df.astype(float)

correction = correction.round(2)
sum_df = sum_df.round(2)

# ทำการเปรียบเทียบ
comparison = correction == sum_df
print("Comparison Result:")
print(comparison)

if comparison.all().all():
    print("All values match!")
else:
    print("There are mismatched values.")

Comparison Result:
   มูลค่าการได้มา  ค่าเสื่อมราคาสะสม  ค่าเผื่อการด้อยค่า  มูลค่าตามบัญชี  \
0            True               True                True            True   
1            True               True                True            True   
2            True               True                True            True   
3            True               True                True            True   
4            True               True                True            True   
5            True               True                True            True   

   ค่าเสื่อมเดือน  ค่าเสื่อมราคาในปี  
0            True               True  
1            True               True  
2            True               True  
3            True               True  
4            True               True  
5            True               True  
All values match!
